In [5]:
import math
import random
from sympy import randprime, isprime, Mod
import hashlib

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [9]:
def GenModulus(w):
    n = len(w) // 2
    p = randprime(2 ** n, 2 ** (n+1))
    q = randprime(2 ** n, 2 ** (n+1))
    N = p * q
    return N, p, q

def randomZnElement(N):
    g = N
    while math.gcd(g, N) != 1:
        g = random.randint(2, N)
    return g

In [62]:
def GenRSA(w):
    n = len(w)
    N, p, q = GenModulus(w)
    m = (p-1) * (q-1)
    e = 2 ** 16 + 1
    d = modinv(e, m)
    return N, e, d, p, q

def enc(x, N, e):
    return x ** e % N

def dec(c, N, d):
    return c ** d % N

In [33]:
def sign(m, N, d):
    sigma = m ** d % N
    return sigma
    
def vrfy(m, s, N, e):
    return s == (m ** e % N)

In [34]:
N, e, d, p, q = GenRSA("11111111")

In [44]:
m = randomZnElement(N)

def h(m, N):
    return (3 * m + 1) % N

In [45]:
s = sign(m, N, d)

print(m, s, N)

vrfy(m, s, N, e)

327 520 667


False

In [32]:
s1 = sign(m1, N, d)
s2 = sing(m2, N, d)

#s = sign(m1 * m2, N, d) = s1 * s2

SyntaxError: cannot assign to function call (<ipython-input-32-899cfa78bcbd>, line 4)

"Regular signatures"

- signer (someone who has a private key needst to learn a meesage that he is about to sign)

"Blind signatures"

- signer (does not know what he is about to sign)


In [46]:
s = sign(m, N, d)

vrfy(m, s, N, e)
print(m, s)

327 520


In [39]:
def blindSgn1(m, e, N):
    # generate blinding factor:
    r = randomZnElement(N)
    return m * (r ** e % N) % N,  r
    
def blindSign2(s, e, N, r):
    rinv = modinv(r, N)
    return s * rinv  % N 

In [47]:
s1, r = blindSgn1(m, e, N)
s2 = sign(s1, N, d)
s = blindSign2(s2, e, N, r)
print(s1, r)
print(s2)
print(m, s)

78 481
662
327 520


In [41]:
vrfy(m, s, N, e)

True

In [54]:
def decSec(c, N, d, e):
    r = randomZnElement(N)
    mp = dec((c * (r ** e % N)) % N, N, d)
    rinv = modinv(r, N)
    return (mp * rinv) % N

In [63]:
c = enc(m, N, e)
d = dec(c, N, d)
print(m, c, d)

327 474 60


In [56]:
c = enc(m, N, e)
d = decSec(c, N, d, e)
print(m, c, d)

327 474 240


In [105]:
N, e, d, p, q = GenRSA("11111111111111111111")
d, N

(1688993, 2040139)

In [106]:
m = random.randint(2, N)
egcd(m, N)

(1, 757197, -230939)

In [107]:
y = enc(m, N, e)
print(y)

831786


In [108]:
x = dec(y, N, d)
print(x, m, x == m)

622226 622226 True


In [109]:
d_bin = "{0:b}".format(d)
print(d, len(d_bin), d_bin)

1688993 21 110011100010110100001


In [120]:
NN, ee, dd, pp, qq = GenRSA("11111111111111111111")
dd_bin = "{0:b}".format(dd)
print(dd, len(dd_bin), dd_bin)

2609737 22 1001111101001001001001


In [ ]:
def fast_pow(c, N, d):
    d_bin = "{0:b}".format(d)
    d_len = len(d_bin)
    reductions = 0
    h = 0
    x = c
    for j in range(1, d_len):
        x, r = mod_reduce(x ** 2, N)
        reductions = reductions + r
        if d_bin[j] == "1":
            x, r = mod_reduce(x * c, N)
            reductions = reductions + r
            #print("A")
            h = h + 1
    return x, h, reductions
    

In [ ]:
def mod_reduce(a, b):
    r = 0
    if a >= b:
        a = a % b
        r = 1
        #print("\tB")
    return a, r

In [ ]:
y = random.randint(2, N)
x, h, r = fast_pow(y, N, d)
h, r

1. y: $ y^3 < N$ ----> if $d[1] == 0/1$ --> no reduction in the first round of the for loop
2. z: $ z^2 < N < z^3$ ---> $d[i] == 1$ --> ther IS a reduction

In [174]:
y = random.randint(2, math.floor(N ** (1/3)))
x, h, r = fast_pow(y, N, d)
h, r

(9, 27)

In [175]:
z = random.randint(math.floor(N ** (1/3)), math.floor(N ** (1/2)))
x, h, r = fast_pow(z, N, d)
h, r

(9, 28)

In [176]:
yy = random.randint(2, math.floor(NN ** (1/3)))
xx, hh, rr = fast_pow(yy, NN, dd)
hh, rr

(10, 29)

In [177]:
zz = random.randint(math.floor(NN ** (1/3)), math.floor(NN ** (1/2)))
xx, hh, rr = fast_pow(zz, NN, dd)
hh, rr

(10, 30)